In [ ]:
!pip install fastapi uvicorn python-multipart pyngrok tensorflow tensorflow-hub opencv-python-headless


In [ ]:
!pip install -q fastapi uvicorn python-multipart pyngrok tensorflow tensorflow-hub opencv-python-headless nest-asyncio

# Imports
import os
import cv2
import numpy as np
import requests
import threading
import tensorflow as tf
import tensorflow_hub as hub
import nest_asyncio
from fastapi import FastAPI, Form
from fastapi.responses import FileResponse
import uvicorn
from pyngrok import ngrok

nest_asyncio.apply()

!ngrok config add-authtoken ngrok_token_here

app = FastAPI()

print("Loading style transfer model...")
hub_model = hub.load('https://tfhub.dev/google/magenta/arbitrary-image-stylization-v1-256/2')
print("Model loaded.")

# Helper: Prepare image for model
def image_read(image_np):
    max_dim = 512
    image = tf.convert_to_tensor(image_np, dtype=tf.float32)
    image = image / 255.0
    shape = tf.cast(tf.shape(image)[:-1], tf.float32)
    long_dim = max(shape)
    scale = max_dim / long_dim
    new_shape = tf.cast(shape * scale, tf.int32)
    image = tf.image.resize(image, new_shape)
    image = image[tf.newaxis, :]
    return image

# Helper: Convert tensor to OpenCV image
def tensor_toimage(tensor):
    tensor = tensor * 255
    tensor = tf.cast(tensor, tf.uint8)
    np_image = tensor.numpy()
    if len(np_image.shape) > 3:
        np_image = np_image[0]
    return cv2.cvtColor(np_image, cv2.COLOR_RGB2BGR)

# Endpoint: Video style transfer
@app.post("/stylize/")
async def stylize_video(style_num: int = Form(...), video_url: str = Form(...)):
    try:
        artworks_dir = "/content/Artworks/"
        os.makedirs(artworks_dir, exist_ok=True)

        style_path = os.path.join(artworks_dir, f"{style_num}.jpg")
        input_path = "/content/input.mp4"
        temp_output_path = "/content/temp_stylized.mp4"
        final_output_path = "/content/output_fixed.mp4"
        print("Looking for style at:", style_path)

        if not os.path.exists(style_path):
            return {"error": f"Style image {style_num}.jpg not found in /content/Artworks."}

        print("Downloading video...")
        video = requests.get(video_url, stream=True)
        with open(input_path, "wb") as f:
            f.write(video.content)

        print(f"Using style image: {style_path}")
        style_image = cv2.imread(style_path)
        style_image = cv2.cvtColor(style_image, cv2.COLOR_BGR2RGB)
        style_image = image_read(style_image)

        cap = cv2.VideoCapture(input_path)
        fps = cap.get(cv2.CAP_PROP_FPS)  # Get FPS from input video
        ret, first_frame = cap.read()
        if not ret:
            raise Exception("Couldn't read video.")
        height, width, _ = first_frame.shape
        out = cv2.VideoWriter(temp_output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))
        cap.set(cv2.CAP_PROP_POS_FRAMES, 0)

        frame_count = 0
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            content_image = image_read(rgb_frame)
            stylized_image = hub_model(tf.constant(content_image), tf.constant(style_image))[0]
            output_frame = tensor_toimage(stylized_image)
            output_frame = cv2.resize(output_frame, (width, height))
            out.write(output_frame)

            frame_count += 1
            if frame_count % 10 == 0:
                print(f"Processed {frame_count} frames...")

        cap.release()
        out.release()
        print("🎞️ Initial style transfer done. Now re-encoding...")

        # Re-encode with H.264 + AAC to ensure Cloudinary compatibility
        os.system(f"ffmpeg -y -i {temp_output_path} -c:v libx264 -preset fast -crf 23 -pix_fmt yuv420p -movflags +faststart {final_output_path}")

        print("Video re-encoded successfully.")
        return FileResponse(final_output_path, media_type="video/mp4", filename="output_fixed.mp4")

    except Exception as e:
        print("ERROR:", str(e))
        return {"error": str(e)}


In [ ]:
!pip install nest_asyncio
import nest_asyncio

nest_asyncio.apply()

In [ ]:
import threading
import uvicorn
from pyngrok import ngrok

# Start FastAPI server in background
def run():
    uvicorn.run(app, host="0.0.0.0", port=8004)


thread = threading.Thread(target=run)
thread.start()

#  Start ngrok tunnel
public_url = ngrok.connect(8004)
print("🚀 Public URL:", public_url)


In [ ]:
# !curl -X POST "https://d62cbe59d2e6.ngrok-free.app/stylize/" -F style_num=16 -F video_url=https://res.cloudinary.com/dfmdmqz4k/video/upload/v1753874203/short_test_bk2mlm.mp4 --output stylized_result.mp4


In [ ]:
!ngrok kill